In [1]:
import json
from web3 import Web3
from solcx import compile_standard, install_solc
import os
from dotenv import load_dotenv

which: no solc in (/home/sahilarora/anaconda3/bin:/home/sahilarora/anaconda3/condabin:/home/sahilarora/.cargo/bin:/usr/local/sbin:/usr/local/bin:/usr/bin:/opt/android-sdk/platform-tools:/home/sahilarora/.dotnet/tools:/var/lib/flatpak/exports/bin:/usr/lib/jvm/default/bin:/usr/bin/site_perl:/usr/bin/vendor_perl:/usr/bin/core_perl)


In [2]:
with open(
    "/home/sahilarora/capstone-19BCE1366-2023/contract/SimpleStorageContract.sol",
    "r",
) as file:
    simple_storage_file = file.read()

In [3]:
install_solc("0.6.0")

Version('0.6.0')

In [4]:
compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"SimpleStorageContract.sol": {"content": simple_storage_file}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version="0.6.0",
)

with open("compiled_code.json", "w") as file:
    json.dump(compiled_sol, file)


In [5]:
bytecode = compiled_sol["contracts"]["SimpleStorageContract.sol"]["SimpleStorage"][
    "evm"
]["bytecode"]["object"]
abi = json.loads(
    compiled_sol["contracts"]["SimpleStorageContract.sol"]["SimpleStorage"]["metadata"]
)["output"]["abi"]

In [6]:
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

In [7]:
chain_id = 1337
my_address = "0x67cC7D1dcAE9b1C23Fc4ef2027d6c1B0981E9B32"
# private_key = os.getenv("PRIVATE_KEY")
private_key = "0x12d8bd1785a48823297579bfa4773015bc7d4de689330fb003338eaf9a707b9f"

In [8]:
SimpleStorage = w3.eth.contract(abi=abi, bytecode=bytecode)

In [9]:
def setupContract():
    nonce = w3.eth.getTransactionCount(my_address)
    tx = SimpleStorage.constructor().buildTransaction(
    {
        "chainId": chain_id,
        "gasPrice": w3.eth.gas_price,
        "from": my_address,
        "nonce": w3.eth.getTransactionCount(my_address),
    }
    )
    signed_tx = w3.eth.account.signTransaction(tx, private_key=private_key)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    print(f"Contract deployed to {tx_receipt.contractAddress}")
    return tx_receipt.contractAddress

In [10]:
contractAddress = setupContract()

Contract deployed to 0x5e1a595a584A4ed638e7c8ca6Bb8c5C83bCcC8d4


In [11]:
def createSupplyNodeTransaction(itemID, itemName, itemICAO, stepID):
    simple_storage = w3.eth.contract(address=contractAddress, abi=abi)
    new_transaction = simple_storage.functions.create_store_event(itemID, itemName, itemICAO, stepID).buildTransaction(
    {
        "chainId": chain_id,
        "gasPrice": w3.eth.gas_price,
        "from": my_address,
        "nonce": w3.eth.getTransactionCount(my_address),
    }
    )
    signed_new_txn = w3.eth.account.sign_transaction(
    new_transaction, private_key=private_key
    )
    tx_new_hash = w3.eth.send_raw_transaction(signed_new_txn.rawTransaction)
    print("Sending new transaction...")
    tx_new_receipt = w3.eth.wait_for_transaction_receipt(tx_new_hash)
    print(tx_new_receipt)
    
    

In [12]:
createSupplyNodeTransaction(1, "n1", "BLR", 2)

Sending new transaction...
AttributeDict({'transactionHash': HexBytes('0x7b7774a8102e5d8feab686942e7807eb314986103ed13b341b00511aece0d7a4'), 'transactionIndex': 0, 'blockNumber': 2, 'blockHash': HexBytes('0xaad626c14d879379bcb4618722868f15e9e9124b44a905a1292deeb2746075c0'), 'from': '0x67cC7D1dcAE9b1C23Fc4ef2027d6c1B0981E9B32', 'to': '0x5e1a595a584A4ed638e7c8ca6Bb8c5C83bCcC8d4', 'cumulativeGasUsed': 135359, 'gasUsed': 135359, 'contractAddress': None, 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [13]:
def printAllContractData():
    simple_storage = w3.eth.contract(address=contractAddress, abi=abi)
    print(f"New stored value at Retrieve {simple_storage.functions.retrieveLog().call()}")

In [14]:
printAllContractData()

New stored value at Retrieve [(1, 'n1', 'BLR', 2)]
